In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

spark = SparkSession.builder.appName("ProductCategory").getOrCreate()

products_data = [
    (1, "Product A"),
    (2, "Product B"),
    (3, "Product C"),
    (4, "Product D"),
]

categories_data = [
    (1, "Category 1"),
    (2, "Category 2"),
    (3, "Category 3"),
]

product_category_data = [
    (1, 1),
    (1, 2),
    (2, 1),
    (3, 3),
]

products_df = spark.createDataFrame(products_data, ["id", "name"])
categories_df = spark.createDataFrame(categories_data, ["id", "name"])
product_category_df = spark.createDataFrame(product_category_data, ["product_id", "category_id"])

def get_product_category_pairs_and_orphan_products(products_df, categories_df, product_category_df):
    product_category_join = products_df.join(
        product_category_df,
        products_df.id == product_category_df.product_id,
        how="left"
    ).join(
        categories_df,
        product_category_df.category_id == categories_df.id,
        how="left"
    ).select(
        products_df.name.alias("product_name"),
        categories_df.name.alias("category_name")
    )
    orphan_products = product_category_join.filter(col("category_name").isNull()).select("product_name").distinct()
    
    return product_category_join, orphan_products

product_category_pairs, orphan_products = get_product_category_pairs_and_orphan_products(products_df, categories_df, product_category_df)

print("Product-Category Pairs:")
product_category_pairs.show()

print("Orphan Products:")
orphan_products.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/22 16:54:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Product-Category Pairs:


+------------+-------------+
|product_name|category_name|
+------------+-------------+
|   Product A|   Category 2|
|   Product A|   Category 1|
|   Product B|   Category 1|
|   Product C|   Category 3|
|   Product D|         NULL|
+------------+-------------+

Orphan Products:


+------------+
|product_name|
+------------+
|   Product D|
+------------+

